In [2]:
import pandas as pd
import statsmodels.api as sm    
import os
import numpy as np

In [3]:
base_path = '/myriadfs/projects/ICS_UKB/projects/kasia_project/causal_analysis/'
file_exposure = os.path.join(base_path, 'group_1_clean_filtered_imputed_dataset_df.tsv') # exposure variables 
file_outcome = os.path.join(base_path, 'group_1_outcomes_df_without_qc_df.tsv') # outcome variables 

exposure_data = pd.read_csv(file_exposure, sep='\t') 
outcome_data = pd.read_csv(file_outcome, sep='\t')

print(f'exposure_data shape: {exposure_data.shape}')
print(f'outcome_data shape: {outcome_data.shape}')

exposure_outcome_data = pd.merge(exposure_data, outcome_data, on='eid', how='inner') # merge exposure and outcome data on 'eid' column
print(f'Merged data shape: {exposure_outcome_data.shape}')

exposure_data shape: (459142, 415)
outcome_data shape: (459142, 8)
Merged data shape: (459142, 422)


In [6]:
df_model = exposure_outcome_data.copy()

raw_cols = [
    '1289-0.0', '1309-0.0', # Diet
    '864-0.0', '884-0.0',   # Activity
    '1050-0.0', '1060-0.0', # Sun
    '1160-0.0',             # Sleep
    '20116-0.0',            # Smoking
    '20117-0.0',            # Alcohol
    '1110-0.0', '2237-0.0', # Electronic
    '2090-0.0', '2040-0.0'  # Mental
]
existing_cols = [i for i in raw_cols if i in df_model.columns] # if column exists in df_model
for col in existing_cols:
    df_model[col] = df_model[col].apply(lambda x: np.nan if x < 0 else x)    

# 1289-0.0: vegetable intake, 1309-0.0: fruit intake
if '1289-0.0' in df_model.columns and '1309-0.0' in df_model.columns: 
    df_model['diet_total'] = df_model['1289-0.0'] + df_model['1309-0.0'] # diet_total = vegetable + fruit    

# 864-0.0: walk activity, 884-0.0: moderate activity
if '864-0.0' in df_model.columns and '884-0.0' in df_model.columns: 
    df_model['activity_is_active'] = ((df_model['864-0.0'] >= 5) | (df_model['884-0.0'] >= 5)).astype(int) # activity_is_active = 1 if walk or moderate activity >= 5 days/week, else 0

# 1050-0.0: summer sun exposure, 1060-0.0: winter sun exposure
if '1050-0.0' in df_model.columns and '1060-0.0' in df_model.columns:
    df_model['sun_exposure_avg'] = (df_model['1050-0.0'] + df_model['1060-0.0']) / 2 # sun_exposure_avg = average of summer and winter sun exposure

# 1160-0.0: sleep duration
if '1160-0.0' in df_model.columns:
    df_model['sleep_hours'] = df_model['1160-0.0']
    # sleep_abnormal = 1 if sleep_hours < 7 or sleep_hours > 8, else 0
    df_model['sleep_abnormal'] = df_model['1160-0.0'].apply(lambda x: 0 if 7 <= x <= 8 else 1)

# 20116-0.0: smoking status
if '20116-0.0' in df_model.columns:
    df_model['smoking_status'] = df_model['20116-0.0'] # smoking_status

# 20117-0.0: alcohol status
if '20117-0.0' in df_model.columns:
    df_model['alcohol_status'] = df_model['20117-0.0'] # alcohol_status    

# electronic device, mental and BMI (keep same as original)
other_cols = {
    'device_mobile':'1110-0.0', # mobile phone use
    'device_computer':'2237-0.0', # computer use
    'mental_doctor':'2090-0.0', # doctor visit
    'mental_risk':'2040-0.0', # risk taking
    'BMI':'21001-0.0' # BMI
}

for new_name, old_name in other_cols.items():
    if old_name in df_model.columns:
        df_model[new_name] = df_model[old_name]

# 'activity_is_active', 'sun_exposure_avg','alcohol_status' missing, so not included in feature_cols for now

feature_cols = ['diet_total',  
                'sleep_hours', 'smoking_status',
                'device_mobile', 'device_computer', 'mental_doctor', 'mental_risk'] # feature columns

print(feature_cols)


['diet_total', 'sleep_hours', 'smoking_status', 'device_mobile', 'device_computer', 'mental_doctor', 'mental_risk']


In [7]:
target_col = 'def_CVD_AFTER'
model_cols = ['age_defined_baseline', 'genetic_sex','BMI'] + feature_cols # age, sex, BMI + feature columns

# clean data for modeling
model_data = df_model[[target_col] + model_cols].dropna()

Y = model_data[target_col] # outcome variable
X = model_data[model_cols] # exposure variables
X = sm.add_constant(X) # add intercept

logit_model = sm.Logit(Y, X).fit() # fit the logistic regression model

print(logit_model.summary())

# calculate odds ratios and confidence intervals
print("\n=== (Odds Ratios) ===")
params = logit_model.params
conf = logit_model.conf_int()
conf['OR'] = params
conf.columns = ['Lower CI', 'Upper CI', 'Odds Ratio']

result_table = np.exp(conf)
result_table['P-value'] = logit_model.pvalues
print(result_table.sort_values(by='P-value',ascending=False))

Optimization terminated successfully.
         Current function value: 0.208296
         Iterations 8
                           Logit Regression Results                           
Dep. Variable:          def_CVD_AFTER   No. Observations:               429116
Model:                          Logit   Df Residuals:                   429105
Method:                           MLE   Df Model:                           10
Date:                Sun, 15 Feb 2026   Pseudo R-squ.:                 0.07287
Time:                        14:04:46   Log-Likelihood:                -89383.
converged:                       True   LL-Null:                       -96408.
Covariance Type:            nonrobust   LLR p-value:                     0.000
                           coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------------
const                   -9.0919      0.085   -106.921      0.000      -9.259      -8.925